# Compute performance metrics for the given Y and Y_score without sklearn

In [2]:
import numpy as np
import pandas as pd
# other than these two you should not import any other packages

<pre>
<font color='red'><b>A.</b></font> Compute performance metrics for the given data <strong>5_a.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points >> number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_a.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a> Note: it should be numpy.trapz(tpr_array, fpr_array) not numpy.trapz(fpr_array, tpr_array)</li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [77]:
# write your code here
a_5 = pd.read_csv("5_a.csv")
print(a_5.head())

     y     proba
0  1.0  0.637387
1  1.0  0.635165
2  1.0  0.766586
3  1.0  0.724564
4  1.0  0.889199


In [78]:
def y_pred(a_5):
    index = 0
    for a in a_5['proba']:
        index = index
        if a >=0.5:
            a_5.loc[index:index,'y_pred'] = 1
        else:
            a_5.loc[index:index,'y_pred'] = 0
        index = index + 1
    return a_5['y_pred']     
y_pred(a_5)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
10095    1.0
10096    1.0
10097    1.0
10098    1.0
10099    1.0
Name: y_pred, Length: 10100, dtype: float64

In [79]:
def confusion_matrix(a_5,y,y_pred):
    TN_0_0=0
    FP_0_1=0
    FN_1_0=0
    TP_1_1=0

    for index, value in enumerate(a_5['y']): 
    
        if a_5.y[index] == 0.0 and a_5.y_pred[index] == 0.0:
            TN_0_0 = TN_0_0 + 1
        elif a_5.y[index] == 0.0 and a_5.y_pred[index] == 1.0:
            FP_0_1 = FP_0_1 + 1
        elif a_5.y[index] == 1.0 and a_5.y_pred[index] == 0.0:
            FN_1_0 = FN_1_0 + 1
        elif a_5.y[index] == 1.0 and a_5.y_pred[index] == 1.0:
            TP_1_1 = TP_1_1 + 1
            
    return TN_0_0, FP_0_1, FN_1_0,  TP_1_1  

TN_0_0, FP_0_1, FN_1_0, TP_1_1  = confusion_matrix(a_5,a_5['y'],a_5['y_pred'])
print(TN_0_0, FP_0_1, FN_1_0, TP_1_1)

0 100 0 10000


In [80]:
def f1_score(TN_0_0,FP_0_1,FN_1_0,TP_1_1):
    precision = TP_1_1/(TP_1_1 + FN_1_0)
    recall = TP_1_1/(TP_1_1 + FP_0_1)
    F1_score = 2*precision*recall/(precision + recall)
    return F1_score
F1_score = f1_score(TN_0_0,FP_0_1,FN_1_0,TP_1_1)
print(F1_score)

0.9950248756218906


In [81]:
from tqdm import tqdm 

def get_tpr_fpr(df):
    tp = ((df['y'] == 1.0 ) & (df['y_pred'] == 1)).sum()
    fp = ((df['y'] == 0.0 ) & (df['y_pred'] == 1)).sum()
    tn = ((df['y'] == 0.0 ) & (df['y_pred'] == 0)).sum()
    fn = ((df['y'] == 1.0 ) & (df['y_pred'] == 0)).sum()

    tpr = tp / (tp + fn )
    fpr = fp / (fp + tn)
    
    return tpr, fpr

def calculated_threshold(df):
    TPR_array = []
    FPR_array = []
    sorted_df = df.sort_values(by='proba', ascending=False)
    unique_sorted_df = sorted_df['proba'].unique()
    for threshold in tqdm(unique_sorted_df):
        sorted_df['y_pred'] = np.where(sorted_df['proba'] >= threshold,1,0)
        tpr,fpr = get_tpr_fpr(sorted_df)
        TPR_array.append(tpr)
        FPR_array.append(fpr)
    return TPR_array, FPR_array
TPR_array, FPR_array = calculated_threshold(a_5)

def AUC(TPR_array, FPR_array):
    auc = np.trapz(TPR_array, FPR_array)
    return auc
AUC(TPR_array, FPR_array)

100%|██████████| 10100/10100 [00:28<00:00, 355.66it/s]


0.48829900000000004

In [82]:
def Accuracy_score(TN_0_0,FP_0_1,FN_1_0,TP_1_1):
    Total_number_point = TN_0_0+ FP_0_1 + FN_1_0 + TP_1_1
    Total_number_of_correctly_classified_point = TN_0_0 + TP_1_1
    try:
        return Total_number_of_correctly_classified_point / Total_number_point 
    except ZeroDivisionError:
        return 0
    
Accuracy_score(TN_0_0,FP_0_1,FN_1_0,TP_1_1)

0.9900990099009901

<pre>
<font color='red'><b>B.</b></font> Compute performance metrics for the given data <strong>5_b.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points << number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_b.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a></li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [83]:
# write your code
b_5 = pd.read_csv("5_b.csv")
b_5.head()

y     proba
0  0.0  0.281035
1  0.0  0.465152
2  0.0  0.352793
3  0.0  0.157818
4  0.0  0.276648

In [84]:
def y_pred(b_5):
    index = 0
    for _ in b_5['y']:
        if b_5.loc[index, 'proba'] >= 0.5:
            b_5.loc[index,'y_pred'] = 1
        else:
            b_5.loc[index,'y_pred'] = 0
        index = index + 1
    return b_5

y_pred(b_5)

y     proba  y_pred
0      0.0  0.281035     0.0
1      0.0  0.465152     0.0
2      0.0  0.352793     0.0
3      0.0  0.157818     0.0
4      0.0  0.276648     0.0
...    ...       ...     ...
10095  0.0  0.474401     0.0
10096  0.0  0.128403     0.0
10097  0.0  0.499331     0.0
10098  0.0  0.157616     0.0
10099  0.0  0.296618     0.0

[10100 rows x 3 columns]

In [91]:
def confusion_matrix(df):
    TN,FP,FN,TP,index=0,0,0,0,0
    for _ in df['y']:
        if df.loc[index,'y'] == 0.0 and df.loc[index,'y_pred'] == 0.0:
            TN = TN + 1
        elif df.loc[index, 'y'] == 0.0 and df.loc[index, 'y_pred'] == 1.0:
            FP = FP + 1
        elif df.loc[index, 'y'] == 1.0 and df.loc[index, 'y_pred'] == 0.0:
            FN = FN + 1
        elif df.loc[index, 'y'] == 1.0 and df.loc[index, 'y_pred'] == 1.0:
            TP = TP + 1
        index = index + 1
    return TN, FP, FN, TP
confusion_matrix(b_5)

(9761, 239, 45, 55)

In [92]:
def f1_score(TN, FP, FN, TP):
    precision = TP/(TP+FN)
    recall = TP/(TP+FP)
    f1_Score = 2*precision*recall / (precision + recall)
    return f1_Score
f1_score(TN, FP, FN, TP)

0.2791878172588833

In [90]:
from tqdm import tqdm

def get_tpr_fpr(df):
    
    tp = ((df['y'] == 1.0 ) & (df['y_pred'] == 1)).sum()
    fp = ((df['y'] == 0.0 ) & (df['y_pred'] == 1)).sum()
    tn = ((df['y'] == 0.0 ) & (df['y_pred'] == 0)).sum()
    fn = ((df['y'] == 1.0 ) & (df['y_pred'] == 0)).sum()

    tpr = tp / (tp + fn )
    fpr = fp / (fp + tn)
    
    return tpr, fpr 
def calculted_threshold(df):
    tpr_array = []
    fpr_array = []
    sorted_df = df.sort_values(by='proba', ascending = False)
    unique_proba_value = sorted_df['proba'].unique()
    for threshold in tqdm(unique_proba_value):
        sorted_df['y_pred'] = np.where(sorted_df['proba']>=threshold,1,0)
        TPR, FPR = get_tpr_fpr(sorted_df)
        tpr_array.append(TPR)
        fpr_array.append(FPR)
    return tpr_array, fpr_array
        
tpr_array, fpr_array = calculted_threshold(b_5)     

def AUC(tpr_array, fpr_array):          
    auc = np.trapz(tpr_array, fpr_array)
    print("AUC SCORE", auc)
AUC(tpr_array, fpr_array)


100%|██████████| 10100/10100 [00:28<00:00, 357.59it/s]

AUC SCORE 0.9377570000000001


In [95]:
def accuacy_score(TN, FP, FN, TP):
    Accuacy_score = (TN + TP) / (TN + FP + FN + TP)
    print("Accuacy_score",Accuacy_score)
accuacy_score(TN, FP, FN, TP)

Accuacy_score 0.9718811881188119


<font color='red'><b>C.</b></font> Compute the best threshold (similarly to ROC curve computation) of probability which gives lowest values of metric <b>A</b> for the given data <strong>5_c.csv</strong>
<br>

you will be predicting label of a data points like this: $y^{pred}= \text{[0 if y_score < threshold  else 1]}$

$ A = 500 \times \text{number of false negative} + 100 \times \text{numebr of false positive}$

<pre>
   <b>Note 1:</b> in this data you can see number of negative points > number of positive points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_c.csv</b>
</pre>

In [4]:
 # write your code
c_5 = pd.read_csv("5_c.csv")
c_5.head(5)

y      prob
0  0  0.458521
1  0  0.505037
2  0  0.418652
3  0  0.412057
4  0  0.375579

In [6]:
y_actual_score = c_5.iloc[:, 0].values
y_prob_score = c_5.iloc[:, 1].values
unique_prob_score = c_5['prob'].unique()

[0.45852068 0.50503693 0.41865174 ... 0.65916054 0.45626546 0.65916054]


In [26]:
def get_matrix(y_actual, y_prob_score, threshold):
    FP, FN, index = 0,0,0
    for _ in y_prob_score:
        if y_prob_score[index]>=threshold and y_actual[index] == 0:
            FP = FP + 1
        elif y_prob_score[index]<threshold and y_actual[index] == 1:
            FN = FN + 1
        index = index + 1
    A = 500 * FN + 100 * FP
    return A
            
def best_threshold(y_actual,y_prob_score, unique_prob_score):
    min_a = float('inf')
    min_t = 0
    for threshold in unique_prob_score:
        a = get_matrix(y_actual, y_prob_score, threshold)
        if a <= min_a:
            min_a = min(min_a,a)
            min_t = threshold
    
    return min_a, min_t

                
print(best_threshold(y_actual_score, y_prob_score, unique_prob_score))

(141000, 0.2300390278970873)


<pre>
<font color='red'><b>D.</b></font> Compute performance metrics(for regression) for the given data <strong>5_d.csv</strong>
    <b>Note 2:</b> use pandas or numpy to read the data from <b>5_d.csv</b>
    <b>Note 1:</b> <b>5_d.csv</b> will having two columns Y and predicted_Y both are real valued features
<ol>
<li> Compute Mean Square Error </li>
<li> Compute MAPE: https://www.youtube.com/watch?v=ly6ztgIkUxk</li>
<li> Compute R^2 error: https://en.wikipedia.org/wiki/Coefficient_of_determination#Definitions </li>
</ol>
</pre>

In [3]:
d_5 = pd.read_csv("5_d.csv")
d_5.head(5)

y   pred
0  101.0  100.0
1  120.0  100.0
2  131.0  113.0
3  164.0  125.0
4  154.0  152.0

In [11]:
n = d_5.shape[0]
print(n)

157200


In [36]:
from sklearn.metrics import mean_squared_error
y_actual_score = d_5.iloc[:, 0].values
y_pred_score = d_5.iloc[:, 1].values
print(mean_squared_error(y_actual_score, y_pred_score))

177.16569974554707


In [6]:
def mse(d_5):
    y_actual_score = d_5.iloc[:, 0].values
    y_pred_score = d_5.iloc[:, 1].values
    diff = y_pred_score - y_actual_score
    diff = diff*diff
    sum_diff = diff.sum()
    average = sum_diff / n
    return average
mse(d_5)

177.16569974554707

In [24]:
def MAPE(d_5):
    y_actual_score = d_5.iloc[:, 0].values
    y_pred_score = d_5.iloc[:, 1].values
    diff = abs(y_pred_score - y_actual_score)
    division = diff / (y_actual_score.sum() / n)
    mape = sum(division) / n
    return mape
    
MAPE(d_5)


0.12912029940096315

In [31]:
def r2_score(d_5):
    y_actual_score = d_5.iloc[:, 0].values
    y_pred_score = d_5.iloc[:, 1].values
    ss_total = y_actual_score - (sum(y_actual_score) / n)
    ss_total = ss_total*ss_total
    ss_total = ss_total.sum()
    ss_res = y_actual_score - y_pred_score
    ss_res = ss_res * ss_res
    ss_res = ss_res.sum()
    r_2_score = 1 -(ss_res/ss_total)
    return r_2_score
r2_score(d_5)

0.9563582786990937

In [32]:
from sklearn.metrics import r2_score
y_actual_score = d_5.iloc[:, 0].values
y_pred_score = d_5.iloc[:, 1].values
print(r2_score(y_actual_score,y_pred_score))

0.9563582786990937
